In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/packer.61/Documents/vote-counts


In [36]:
import pandas as pd
from pathlib import Path
import numpyro
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from src import preprocessing
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [42]:
def get_pivot_odds(df):
    for ind, row in df.iterrows():
        mean_p = row["p_democrat"]
        std_p = row["std_p_democrat"]
        df.loc[ind, "p_05"] = stats.t.pdf(0.5, 4, loc=mean_p, scale=std_p)

        
    df["pivot_odds"] = df["p_05"] / df["total_votes"]
    df["log_pivot_odds"] = np.log(df["p_05"] / df["total_votes"])
    return df

In [25]:
raw_data = pd.read_csv(path_raw_data / "President_2020.csv")
raw_data["party_simplified"] = raw_data["party_simplified"].astype(str)
raw_data["jurisdiction_fips"] = raw_data["jurisdiction_fips"].dropna().astype(int)
raw_data = raw_data.query("party_simplified == 'REPUBLICAN' or party_simplified == 'DEMOCRAT'")

/tmp/ipykernel_21593/1280772690.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "President_2020.csv")


In [43]:
groupby = ["jurisdiction_fips", "party_simplified"]
partisan_votes_by_jurisdiction = (
    raw_data[[*groupby, "votes"]]
    .groupby(groupby)
    .sum()
    .reset_index()
    .pivot_table(values="votes", columns="party_simplified", index="jurisdiction_fips")
)
partisan_votes_by_jurisdiction["total_votes"] = (
    partisan_votes_by_jurisdiction["DEMOCRAT"]
    + partisan_votes_by_jurisdiction["REPUBLICAN"]
)
partisan_votes_by_jurisdiction["p_democrat"] = (
    partisan_votes_by_jurisdiction["DEMOCRAT"]
    / partisan_votes_by_jurisdiction["total_votes"]
)

# Setting a fixed std, should be determined by samples
std = 0.04
partisan_votes_by_jurisdiction["std_p_democrat"] = std

In [45]:
partisan_votes_by_jurisdiction = get_pivot_odds(partisan_votes_by_jurisdiction)

/home/packer.61/mambaforge/envs/pystats/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [55]:
to_save = partisan_votes_by_jurisdiction[["pivot_odds", "log_pivot_odds"]]
to_save.index = to_save.index.astype(int).to_series().apply(lambda i : str(i).zfill(5))
to_save.to_csv(path_data / "partisan_county_pivot_odds.csv")

In [54]:
to_save.index.to_series().apply(lambda i: str(i).zfill(5))

jurisdiction_fips
1001               01001
1003               01003
1005               01005
1007               01007
1009               01009
                 ...    
5514173475    5514173475
5514173900    5514173900
5514182650    5514182650
5514188200    5514188200
5514188575    5514188575
Name: jurisdiction_fips, Length: 7935, dtype: object